In [201]:
import utils.common
import evaluation
import importlib
import numpy as np
import tensorflow as tf
import models.MatrixFactorization
import models.BPR
import models.ConstraintAutoRec
import models.NeuralLogicRec
import pandas as pd
from models.NeuralLogicRec import item_cf, user_cf, diversity_constraint, Constraint, And, Or, Implies, Forall, Not, Equiv
importlib.reload(utils.common)
importlib.reload(evaluation)
importlib.reload(models.NeuralLogicRec)

<module 'models.NeuralLogicRec' from '/home/ec2-user/SageMaker/rs-ml-with-constraints/models/NeuralLogicRec.py'>

In [202]:
ev_test = evaluation.Evaluation(utils.common.ml_small)

In [203]:
constraints = list()
# constraints.append(Constraint(weight=0.4, formula=item_cf))
# constraints.append(Constraint(weight=1.0, formula=user_cf))
@tf.function
def likes_equiv(model, outputs):
    return Forall(Equiv(outputs['rec'], outputs['likes']))
constraints.append(Constraint(weight=0.75, formula=likes_equiv))
@tf.function
def novelty_constraint(model, outputs):
    return Forall(Implies(outputs['popular'], Not(outputs['rec'])))
constraints.append(Constraint(weight=0.25, formula=novelty_constraint))
constraints.append(Constraint(weight=0.25, formula=diversity_constraint))

In [204]:
nlr = models.NeuralLogicRec.NLR(utils.common.ml_small['user'], utils.common.ml_small['dimensions'], epochs=1, constraints=constraints, mode='ae')
nlr.train(utils.common.load_dataset(utils.common.ml_small), utils.common.ml_small['train']['records'])

ValueError: Num gradients 0 generated for op name: "PartitionedCall"
op: "PartitionedCall"
input: "Maximum"
attr {
  key: "Tin"
  value {
    list {
      type: DT_FLOAT
    }
  }
}
attr {
  key: "Tout"
  value {
    list {
      type: DT_FLOAT
    }
  }
}
attr {
  key: "_gradient_op_type"
  value {
    s: "PartitionedCall-4860936"
  }
}
attr {
  key: "config"
  value {
    s: ""
  }
}
attr {
  key: "config_proto"
  value {
    s: "\n\007\n\003GPU\020\001\n\007\n\003CPU\020\0012\005*\0010J\0008\001"
  }
}
attr {
  key: "executor_type"
  value {
    s: ""
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_IsEqual_4860935"
    }
  }
}
 do not match num inputs 1